## Import libraries

In [74]:
import os, io, json
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError


## Set up environmental variables

In [99]:
# Modify this line to include all AOI folder names
AOI_lst = ['GIF14_Au', 'isocronas_las_tablas']

In [94]:
root_folder = os.getcwd().split('\\')
root_folder = root_folder[1:-3]
if root_folder[-1] != 'fireRunSeverity':
    print("Didn't get correct root folder!")
    print(root_folder)
root_folder = os.path.join('C:\\', *root_folder)

os.path.basename(root_folder)

'fireRunSeverity'

## Start Connection with Google Earth Engine

In [80]:
# Set up the Drive API client
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]
creds = None

In [83]:
# The file 'credentials.json' should be downloaded from Google Cloud Console
cred_js_path = os.path.join(root_folder, 'Python/geeFetch/.credential.json')
token_js_path = os.path.join(root_folder, 'Python/geeFetch/.token.json')

# Check if `SCOPES` is modified
with open(token_js_path, 'r') as file:
    data = json.load(file)
if SCOPES != data['scopes']:
    print("SCOPES changed!")
    os.remove(token_js_path)

# Set up connection token or regenerate a new one
if os.path.exists(token_js_path):
    creds = Credentials.from_authorized_user_file(token_js_path, SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          cred_js_path, SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(token_js_path, "w") as token:
      token.write(creds.to_json())

SCOPES changed!
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=375803039104-6qihieo49bltefni8tc36ih8db7c8951.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57912%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=YfvMHlECcndvmF1z7yK1UhJsPi4Sd5&access_type=offline


In [93]:
try:
    service = build("drive", "v3", credentials=creds)

    # Call the Drive v3 API
    results = (
        service.files()
        .list(pageSize=10, fields="nextPageToken, files(id, name)")
        .execute()
    )
    items = results.get("files", [])

    if not items:
      print("No files found.")
      exit()
    print("Files:")
    
    # Match file in the selected AOI
    items_filt = []
    for item in items:
      if item['name'].split("--")[0] == "PythonGEE_output":
         print(f"{item['name']} ({item['id']})")
         items_filt.append(item)
except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f"An error occurred: {error}")

Files:
PythonGEE_output--GIF14_Au--S2_dNBR--19012025_2048.tif (1C6cHjzE2zNTPY16H5VSuEQ-DhCg5rwke)


## Download to local folder

In [100]:
for in_Name in AOI_lst:
    print("#------------------------------#")
    print("AOI:", in_Name)
    for item in items_filt:
        if item['name'].split("--")[1] == in_Name:
            ### Folder ### 
            # Check Folder exists
            outGEEFLD = os.path.join(root_folder, 'data', 'GEE', in_Name)
            if os.path.isdir(outGEEFLD):
                print('Folder', in_Name, 'exist!')
                pass
            else:
                os.mkdir(outGEEFLD)

            ### Connection ###    
            # Request the file metadata to get the name
            file = service.files().get(fileId=item['id']).execute()
            print("Found online file:", file['name'])

            ### Download ###
            # Download the file
            out_file_name = "--".join(items_filt[0]['name'].split("--")[2:]) # delete AOI recognizing string
            out_file_name = os.path.join(outGEEFLD, out_file_name)
            if os.path.exists(out_file_name):
                print('!--', os.path.basename(out_file_name), 'exists!')
                continue
            else:
                request = service.files().get_media(fileId=item['id'])
                fh = io.FileIO(out_file_name, 'wb')
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")
    print("#------------------------------#\n\n")


#------------------------------#
AOI: GIF14_Au
Folder GIF14_Au exist!
Found online file: PythonGEE_output--GIF14_Au--S2_dNBR--19012025_2048.tif
!-- S2_dNBR--19012025_2048.tif exists!
#------------------------------#


#------------------------------#
AOI: isocronas_las_tablas
#------------------------------#


